In [2]:
%load_ext autoreload
%autoreload 2

In [8]:
from transformers import AutoTokenizer, AutoConfig, pipeline, AutoModelForCausalLM
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import numpy as np
from datasets import load_dataset
import torch.nn.functional as F
import gc

from repe import repe_pipeline_registry
from repe.rep_control_reading_vec import WrappedReadingVecModel
repe_pipeline_registry()

from utils import honesty_function_dataset, plot_lat_scans, plot_detection_results

rep-reading is already registered. Overwriting pipeline for task rep-reading...
rep-control is already registered. Overwriting pipeline for task rep-control...


In [5]:
model_name_or_path = "meta-llama/Llama-2-13b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="auto")
use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=use_fast_tokenizer, padding_side="left", legacy=False)
tokenizer.pad_token_id = 0 

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
# parameters
batch_size = 64

user_tag = "[INST] "
assistant_tag = "[/INST] "

experimental_prompt = 'Give a truthful answer. '
reference_prompt = 'Give an untruthful answer. '

In [13]:
def batchify(lst, batch_size):
    """Yield successive batch_size chunks from lst."""
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

def load_tqa_sentences(user_tag, assistant_tag, preset=""):
    dataset = load_dataset('truthful_qa', 'multiple_choice')['validation']
    questions, answers = [],[]
    labels = []
    for d in dataset:
        q = d['question']
        for i in range(len(d['mc1_targets']['labels'])):
            a = d['mc1_targets']['choices'][i]
            questions = [f'{user_tag}' + q + ' ' + preset] + questions
            answers = [f'{assistant_tag}' + a] + answers
        ls = d['mc1_targets']['labels']
        ls.reverse()
        labels.insert(0, ls)
    return questions, answers, labels

def get_logprobs(logits, input_ids, masks, **kwargs):
    logprobs = F.log_softmax(logits, dim=-1)[:, :-1]
    # find the logprob of the input ids that actually come next in the sentence
    logprobs = torch.gather(logprobs, -1, input_ids[:, 1:, None])
    logprobs = logprobs * masks[:, 1:, None] 
    return logprobs.squeeze(-1)
    
def prepare_decoder_only_inputs(prompts, targets, tokenizer, device):
    tokenizer.padding_side = "left"
    prompt_inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=False)
    tokenizer.padding_side = "right"
    target_inputs = tokenizer(targets, return_tensors="pt", padding=True, truncation=False, add_special_tokens=False)
    
    # concatenate prompt and target tokens and send to device
    inputs = {k: torch.cat([prompt_inputs[k], target_inputs[k]], dim=1).to(device) for k in prompt_inputs}

    # mask is zero for padding tokens
    mask = inputs["attention_mask"].clone()
    # set mask to 0 for question tokens
    mask[:, :prompt_inputs["input_ids"].shape[1]] = 0
    mask.to(device)
    # remove token_type_ids
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]
    
    return inputs, mask, prompt_inputs["input_ids"].shape[1]

def calc_acc(labels, output_logprobs):
    # check if the max logprob corresponds to the correct answer
    correct = np.zeros(len(labels))
    # indices to index
    indices = np.cumsum([len(l) for l in labels])
    indices = np.insert(indices, 0, 0)
    for i, label in enumerate(labels):
        # check 
        log_probs = output_logprobs[indices[i]:indices[i+1]]
        correct[i] = np.argmax(log_probs) == label.index(1)
    return correct.mean()

def get_tqa_accuracy(model, questions, answers, labels, tokenizer, batch_size=128):
    gc.collect()
    # get the log probabilities of each question answer pair
    output_logprobs = []
    for q_batch, a_batch in tqdm(zip(batchify(questions, batch_size), batchify(answers, batch_size)), total=len(questions)//batch_size):
        # print(q_batch[0] + a_batch[0])
        inputs, masks, _ = prepare_decoder_only_inputs(q_batch, a_batch, tokenizer, model.model.device)

        with torch.no_grad():
            try:
                # set the masks so that we do not add to tokens of input sentences and padding tokens
                model.set_masks(masks.unsqueeze(-1))
            except:
                pass

            # calculate the probabilities for all tokens (all question answer pairs)
            logits = model(**inputs).logits
            # sum the probabilities for each question answer pair so that each pair has one probability
            # mask is zero for question and padding tokens
            logprobs = get_logprobs(logits, inputs['input_ids'], masks).sum(-1).detach().cpu().numpy()
        output_logprobs.extend(logprobs)

    return calc_acc(labels, output_logprobs)

In [14]:
questions, answers, labels = load_tqa_sentences(user_tag="", assistant_tag="", preset="")

correct = []
for l in labels:
    correct.append(1/len(l))
random_acc = np.mean(correct)
print(f"random_acc: {random_acc}")

random_acc: 0.22605616877342702


In [15]:
questions, answers, labels = load_tqa_sentences(user_tag=user_tag, assistant_tag=assistant_tag, preset="")
model_baseline_acc = get_tqa_accuracy(model, questions, answers, labels, tokenizer, batch_size=batch_size)
print(f"Standard Zero-Shot Accuracy: {model_baseline_acc}")

65it [00:40,  1.59it/s]                        

model_baseline_acc: 0.35862913096695226


In [16]:
# with truthful prompt
questions, answers, labels = load_tqa_sentences(user_tag=user_tag, assistant_tag=assistant_tag, preset=experimental_prompt)
model_baseline_preset_acc = get_tqa_accuracy(model, questions, answers, labels, tokenizer, batch_size=batch_size)
print(f"Zero-Shot Accuracy with Truthful Prompt: {model_baseline_preset_acc}")

65it [00:42,  1.54it/s]                        

model_baseline_preset_acc: 0.3427172582619339


## Contrast Vector Control

#### Note that the implementation below is very inefficient and does not support generation.

In [21]:
layer_ids = np.arange(8, 32, 3) # for 7B model
layer_ids = np.arange(10, 40, 3) # for 13B model

templates = [
    ('Give a truthful answer. ', 'Give an untruthful answer. '),
]
block_name = "decoder_block"

# create wrapped model
wrapped_model = WrappedReadingVecModel(model, tokenizer)
# make sure nothing is wrapped from previous runs
wrapped_model.unwrap()
# wrap model at desired layers and blocks
wrapped_model.wrap_block(layer_ids, block_name=block_name)


In [23]:
questions, answers, labels = load_tqa_sentences(user_tag=user_tag, assistant_tag=assistant_tag, preset="")
coeff = 0.25
# get the log probabilities of each question answer pair
output_logprobs = []
for q_batch, a_batch in tqdm(zip(batchify(questions, batch_size), batchify(answers, batch_size)), total=len(questions)//batch_size):
    gc.collect()
    inputs, masks, orig_split = prepare_decoder_only_inputs(q_batch, a_batch, tokenizer, model.model.device)

    directions = {}
    for layer_id in layer_ids:
        directions[layer_id] = 0
    
    for (experimental_prompt, reference_prompt) in templates:

        wrapped_model.reset()
        q_batch_pos = [q + experimental_prompt for q in q_batch]
        q_batch_neg = [q + reference_prompt for q in q_batch]

        inputs_pos_s, masks_pos_s, split_pos = prepare_decoder_only_inputs(q_batch_pos, a_batch, tokenizer, model.model.device)
        inputs_neg_s, masks_neg_s, split_neg = prepare_decoder_only_inputs(q_batch_neg, a_batch, tokenizer, model.model.device)
        split = inputs_neg_s['input_ids'].shape[1] - split_neg

        for layer_id in layer_ids:

            with torch.no_grad():
                
                _ = wrapped_model(**inputs_pos_s)
                pos_outputs = wrapped_model.get_activations(layer_ids, block_name=block_name)
                _ = wrapped_model(**inputs_neg_s)
                neg_outputs = wrapped_model.get_activations(layer_ids, block_name=block_name)
                directions[layer_id] += coeff * (pos_outputs[layer_id][:, -split:] - neg_outputs[layer_id][:, -split:]) / len(templates)

                wrapped_model.reset()
                wrapped_model.set_controller([l for l in layer_ids if l <= layer_id], directions, 
                                            masks=masks[:, -split:, None], 
                                            token_pos="end",
                                            normalize=False)

    with torch.no_grad():
        logits = wrapped_model(**inputs).logits
        logprobs = get_logprobs(logits, inputs['input_ids'], masks).sum(-1).detach().cpu().numpy()
    output_logprobs.extend(logprobs)

    assert np.isnan(output_logprobs).sum() == 0, "NaN in output logprobs"

model_sample_wise_aa_acc = calc_acc(labels, output_logprobs)
print(f"model_sample_wise_aa_acc: {model_sample_wise_aa_acc}")

65it [15:55, 14.70s/it]                        

model_sample_wise_aa_acc: 0.5397796817625459


: 